In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [4]:
# read the data
data_path = 'dataset_practica/'
good_data = pd.read_csv(data_path + '2000_goodware_2018_2020_features.csv')
# create new column to indicate the class
good_data['class'] = 0
bad_data = pd.read_csv(data_path + '2000_malware_2018_2020_features.csv')
bad_data['class'] = 1

In [5]:
good_data.head()

,hash,PackageName,min_andrversion,max_andrversion,target_andrversion,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,...,TextViewCount,EditViewCount,ImageButtonCount,CheckBoxCount,RadioGroupCount,RadioButtonCount,ToastCount,SpinnerCount,ListViewCount,class
0,000c6c8ef16e3793674bf1bf2cb0cfdb,com.gf.client,17,0,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8e46f1e9830d46402495c0450c489673,com.blueframetech.apexsports,21,0,28,0,0,0,0,0,...,66,3,3,2,2,3,0,40,54,0
2,b28937c14945eef354bbb893f3a97c74,com.devgame66.sniperking,17,0,28,0,0,0,0,0,...,50,3,3,2,0,3,0,37,27,0
3,07c7bc0ea194ff996c4c653b6a10432a,com.auroracoopapp,26,0,29,0,0,0,0,0,...,64,11,14,19,1,16,0,48,54,0
4,334f46c3aeb57318f7a91194cc03f407,com.artha1wides.pkbm_c_new,14,0,28,0,0,0,0,0,...,50,3,3,3,0,3,1,46,43,0


In [6]:
bad_data.head()

,hash,PackageName,min_andrversion,max_andrversion,target_andrversion,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,...,TextViewCount,EditViewCount,ImageButtonCount,CheckBoxCount,RadioGroupCount,RadioButtonCount,ToastCount,SpinnerCount,ListViewCount,class
0,367a147343eb9af9c94cddab76d7c246,com.qimacode.signmaster,16,0,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,f23bf535a96d1ccc5289749069592f42,com.wanimal.travel,19,0,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,e162763edf3c9ef1071997916e0f6c32,com.proxy.cctv28_9,17,0,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3603035190703aac2851e5c8391e302c,com.proguard.camera.fabric.beauty,16,0,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,483b8c48184410151a4089e9101a2429,com.hopeway.cam.p2pcamplus3,16,0,28,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# mix all data into one
data = pd.concat([good_data, bad_data], axis=0)

In [8]:
data.head()

,hash,PackageName,min_andrversion,max_andrversion,target_andrversion,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,...,TextViewCount,EditViewCount,ImageButtonCount,CheckBoxCount,RadioGroupCount,RadioButtonCount,ToastCount,SpinnerCount,ListViewCount,class
0,000c6c8ef16e3793674bf1bf2cb0cfdb,com.gf.client,17,0,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8e46f1e9830d46402495c0450c489673,com.blueframetech.apexsports,21,0,28,0,0,0,0,0,...,66,3,3,2,2,3,0,40,54,0
2,b28937c14945eef354bbb893f3a97c74,com.devgame66.sniperking,17,0,28,0,0,0,0,0,...,50,3,3,2,0,3,0,37,27,0
3,07c7bc0ea194ff996c4c653b6a10432a,com.auroracoopapp,26,0,29,0,0,0,0,0,...,64,11,14,19,1,16,0,48,54,0
4,334f46c3aeb57318f7a91194cc03f407,com.artha1wides.pkbm_c_new,14,0,28,0,0,0,0,0,...,50,3,3,3,0,3,1,46,43,0


In [9]:
# check which columns are numerical and make a list of them
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
# keep only the numerical columns
data = data[numerical_columns]

In [10]:
data.head()

,min_andrversion,max_andrversion,target_andrversion,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,android.permission.ACCESS_DOWNLOAD_MANAGER,android.permission.ACCESS_DOWNLOAD_MANAGER_ADVANCED,...,TextViewCount,EditViewCount,ImageButtonCount,CheckBoxCount,RadioGroupCount,RadioButtonCount,ToastCount,SpinnerCount,ListViewCount,class
0,17,0,27,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,21,0,28,0,0,0,0,0,0,0,...,66,3,3,2,2,3,0,40,54,0
2,17,0,28,0,0,0,0,0,0,0,...,50,3,3,2,0,3,0,37,27,0
3,26,0,29,0,0,0,0,0,0,0,...,64,11,14,19,1,16,0,48,54,0
4,14,0,28,0,0,0,0,0,0,0,...,50,3,3,3,0,3,1,46,43,0


In [11]:
device = 'cuda'

In [12]:
malware_sample = data[data['class'] == 1].iloc[0]
malware_features = torch.tensor(malware_sample[:-1].values).float().unsqueeze(0).to(device)

In [13]:
class MalwareClassifier(nn.Module):
    def __init__(self):
        super(MalwareClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(malware_features.shape[1], 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),                
            nn.Dropout(0.5),                   

            nn.Linear(256, 128),              
            nn.ReLU(),
            nn.BatchNorm1d(128),             
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

In [15]:
# Load the pre-trained model
model_path = 'malware_classifier.pth'
model = MalwareClassifier()
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

MalwareClassifier(
  (network): Sequential(
    (0): Linear(in_features=361, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=64, out_features=1, bias=True)
    (13): Sigmoid()
  )
)

In [16]:
with torch.no_grad():
    prediction = model(malware_features)
    predicted_class = prediction.round().item()
    print("Predicted Class:", "Malware" if predicted_class == 1 else "Goodware")

Predicted Class: Malware


In [19]:
modified_features = malware_features.clone().to(device)
perturbation = torch.randn(modified_features.shape) * 0.1
perturbation = perturbation.to(device)
modified_features += perturbation
modified_features = torch.clamp(modified_features, 0, 1)

In [20]:
# Test the modified sample
with torch.no_grad():
    modified_prediction = model(modified_features)
    modified_predicted_class = modified_prediction.round().item()
    print("Modified Predicted Class:", "Malware" if modified_predicted_class == 1 else "Goodware")

Modified Predicted Class: Goodware
